### This notebook shows the core of the proposed framework: evaluation of ANN classifiers with latent space performance metrics

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from latentspace.ml_util import *
import latentspace.datasets as datasets
import latentspace.generative as generative
import latentspace.cnn as cnn
from latentspace.adversarial_generation import *
from latentspace.evaluation_util import EvaluationUtil

### Load a dataset and classifiers

In [ ]:
dataset_info = DatasetInfo.ImageNetAnimals
no_classes = 3
no_subclasses = 5
ds = datasets.ImageNetAnimalsData()
classifier_d = "animals-128"
gm = generative.BigGAN(ds)
class_proportions = np.repeat(1 / no_classes, no_classes)
unit_type = 0
    
def load_classifier(prefix: str):
    weights_filename = f"classifiers_architecture{unit_type}/{prefix}_{dataset_info.name}.bin"
    c = cnn.Trainer(classifier_d, ds.get_train_loader, ds.get_test_loader, unit_type=unit_type)
    c.restore_params_from_disk(weights_filename)
    return c

nonrobust_classifier = load_classifier("plain")
robust_classifier    = load_classifier("robust")
classifiers = [nonrobust_classifier, robust_classifier]

def advgen_experiments(adversary: Adversary, noise_eps: float, total_no_images: int):
    decay_factor = EpsDTransformer().eps_to_d(noise_eps)
    advgen = AdversarialGenerator(None, classifiers, True, decay_factor)
    advgen.set_generative_model(gm)
    no_images = np.round(class_proportions * total_no_images)
    for i in range(no_classes):
        for subclass_index in range(no_subclasses):
            LogUtil.info(f"*** {classifier_d.upper()}, CLASS {i}, SUBCLASS {subclass_index} ***")
            LogUtil.info(f"noise_epsilon = {noise_eps:.5f}, decay_factor = {decay_factor:.5f}")
            gm.configure_class(i, subclass_index)
            advgen.generate(adversary, int(no_images[i]), True, clear_stat=(i == 0))
    LogUtil.info("*** STATISTICS ***")
    advgen.print_stats(True)

### Quick demonstration how generative models and classifiers work

* For each class, show image reconstructions (as pairs original - reconstructed) and generated images

In [ ]:
for i in range(no_classes):
    for subclass_index in range(no_subclasses):
        LogUtil.info(f"*** {classifier_d.upper()}, CLASS {i}, SUBCLASS {subclass_index} ***")
        gm.configure_class(i, subclass_index)
        EvaluationUtil.show_generated_images(gm, 1, 8)

### Measure local latent noise accuracy (LLNA)

* Also show noise-based perturbations if requested
* Change the numbers of images to get more meaningful results

In [ ]:
no_images = 1
no_perturbations_per_image = 1
visualize = True
for i in range(no_classes):
    for subclass_index in range(no_subclasses):
        LogUtil.info(f"*** {classifier_d.upper()}, CLASS {i}, SUBCLASS {subclass_index} ***")
        gm.configure_class(i, subclass_index)
        RandomPerturbationStatistician(gm, classifiers, no_images, no_perturbations_per_image,
                                       visualize, np.linspace(0.25, 1.00, 4)).process()

### Measure latent adversarial generation severity (LAGS) by searching for minimum latent adversarial perturbations

* Change the number of images to get more meaningful results
* Metric reports are after each series of reconstructed/generated images
* Latent generation accuracy (LGA) is also computed
* Norms of minimum latent adversarial perturbations in the original space are also computed

In [ ]:
total_images = 10
max_rho, noise_eps = 2.5, 1.0
adversary = PGDAdversary(max_rho, 50, 0.05, False, 0, verbose=0)   
advgen_experiments(adversary, noise_eps, total_images)

### Measure latent adversarial generation accuracy (LAGA) by searching for bounded latent adversarial perturbations

* Change the number of images to get more meaningful results
* Metric reports are after each series of reconstructed/generated images
* Latent generation accuracy (LGA) is also computed

In [ ]:
total_images = 10
rho, noise_eps = 0.3, 1.0
# search with restarts (a sequence of restarts will terminate if an adversarial perturbation is found)
adversary = PGDAdversary(rho, 50, 0.05, True, 0, verbose=0, n_repeat=12, repeat_mode="any")
advgen_experiments(adversary, noise_eps, total_images)